In [27]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import brown
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict

nltk.download('brown')

# helper functions

In [41]:
def read_file(loc):
    with open(loc) as f:
        return [x.strip() for x in f.readlines()]

In [49]:
def fill_df(df, terms):
    # count number of example sentences (unverified) for each example
    terms_dict = defaultdict(list)
    for item in tqdm(terms):
        if item in df.columns:
            continue
        for sent in df['sent']:
            sent = ' '.join(sent)
            if item + ' ' in sent:
                terms_dict[item].append(1)
            else:
                terms_dict[item].append(0)
        df[item] = terms_dict[item]
    return df

In [ ]:
def clean_sents(df):
    # provide 'clean' version of sentence, without the relevant term
    # checks for capitalization and makes proper if necessary
    return

# discourse markers: Brown corpus

In [59]:
brown_disc_df = pd.read_pickle('data/discourse_markers/brown_disc_df.pkl')

In [39]:
brown_disc_df.to_pickle('data/discourse_markers/brown_disc_df.pkl')

# qualifiers/intensifiers: Brown corpus

In [60]:
qual = read_file('data/qualifiers_intensifiers/qual_intens_list.txt')

In [62]:
brown_qual_df = pd.DataFrame()
brown_qual_df['sent'] = sents

In [63]:
brown_qual_df = fill_df(brown_qual_df, [' ' + x for x in qual])

In [64]:
brown_qual_df.head()

,sent,quite,simply,surely,just,actually,please,exact,then,literally,...,ideally,relatively,maybe,far,clearly,too,completely,very,honestly,sorely
0,"[The, Fulton, County, Grand, Jury, said, Frida...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[The, jury, further, said, in, term-end, prese...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[The, September-October, term, jury, had, been...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[``, Only, a, relative, handful, of, such, rep...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[The, jury, said, it, did, find, that, many, o...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
brown_qual_df.columns = [x[1:] for x in brown_qual_df.columns]

In [70]:
brown_qual_df.head()

,ent,quite,simply,surely,just,actually,please,exact,then,literally,...,ideally,relatively,maybe,far,clearly,too,completely,very,honestly,sorely
0,"[The, Fulton, County, Grand, Jury, said, Frida...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[The, jury, further, said, in, term-end, prese...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[The, September-October, term, jury, had, been...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[``, Only, a, relative, handful, of, such, rep...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[The, jury, said, it, did, find, that, many, o...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# count number of example sentences (unverified) for each example
for item in tqdm(qual):
    count = 0
    for sent in sents:
        sent = ' '.join(sent)
        if ' ' + item + ' ' in sent:
            count += 1
    print(item + '\t' + str(count))

quite	269
simply	166
surely	38
just	742
actually	127
please	45
exact	27
then	984
literally	26
a lot	85
of course	234
really	267
probably	232
sort of	117
incredibly	7
some	1345
usually	185
well	744
all of	153
blatantly	0
er	0
much	878
even	954
occasionally	32
also	983
ultimately	19
if ever	3
certainly	115
a bit	56
perfectly	31
importantly	8
definitely	21
necessarily	49
alone	190
pretty	98
some kind of	21
truly	56
possibly	57
fully	80
extremely	50
vast	60
little	767
all	2611
so	1641
some	1345
ever	328
so-called	31
right	577
big	308
at least	272
much	878
lots of	26
particularly	141
specifically	36
at all	183
such	1124
apparently	102
absolutely	27
exactly	99
in effect	24
actively	11
generally	119
admittedly	3
ideally	5
relatively	84
maybe	66
far	401
clearly	118
too	760
completely	109
very	749
well	744
all of	153
honestly	12
much	878
sorely	3
